# Visualizing Sustainability Insights using NYC Water and Energy Data
In the previous project, we saw that NYC water consumption has decreased significantly over the past four decades in both absolute and per-capita quantities. However we also learned that the rate of improvement in conservation has slowed down.

Your project now is to identify focus areas for the next water conservation campaign by looking in detail at consumption records that large buildings in NYC have provided each year since 2010. You will look for large water usage by considering the location (borough), building use, occupancy, size and electricity usage.

In our previous project, we were presented with a data set that was easily visualized - all the columns were relevant and it had clean data.  In this project, we're going to work with more "raw" data sets that will need to be cleaned, combined and then transformed to produce visualizations that can help us answer the following questions:

- Is there a relationship between electricity consumption and water consumption? 
    - Is there a general correlation?
    - Are there specific correlations based on building type?
- Which categories of buildings consume the most water?
    - Is there a significant difference in consumption per building type based on the borough it is in?
    - For these categories, how has their consumption changed in the past decade? 
- Based on your analysis, where do you think the city should focus its conservation efforts?

> If you are not already in a Jupyter notebook environment, you can launch this notebook in an online JupyterLab session hosted by mybinder.org:
> 
> https://mybinder.org/v2/gh/intersective/binder-base/trunk?urlpath=git-pull%3Frepo%3Dhttps%253A%252F%252Fgithub.com%252Fintersective%252Fdata-capstones%26urlpath%3Dlab%252Ftree%252Fdata-capstones%252Fskillsbuild%252Fsustainability%252Fnyc_water_project_2.ipynb%26branch%3Dtrunk

First, let's set up our workbook with the necessary python libraries.

In [ ]:
import pandas as pd
import numpy as np
import nbformat
import cufflinks as cf
import plotly.express as px
import plotly.offline as py
import plotly.graph_objs as go
cf.go_offline() # required to use plotly offline (no account required).
py.init_notebook_mode() # graphs charts inline (IPython).

Now let's access our data source. NYC's Open Data initiative makes data sets available via their simple SOCRATA API system. You can get all of the data in JSON format, or you can pre-filter the data.

This data set has over 150 columns, most of which are not needed for this project. We've selected a subset of the columns to start with, which will make obtaining the data more efficient. Let's print out the API URL - we can click on this and see the raw data. Let's also look at what data types python thinks we have for each column.

In [ ]:
data2013 = 'https://data.cityofnewyork.us/resource/yr5p-wjer.json'
desiredColumns13 = 'property_id,year_ending,borough,postcode,occupancy,primary_property_type_epa,water_use_all_water_sources,indoor_water_use_all_water,site_energy_use_kbtu'
data2022 = 'https://data.cityofnewyork.us/resource/7x5e-2fxh.json'
desiredColumns22 = 'property_id,year_ending,borough,property_gfa_calculated,postal_code,occupancy,primary_property_type,water_use_all_water_sources,indoor_water_use_all_water,site_energy_use_kbtu'
dataUrl = data2022 + '?$limit=60000&$select=' + desiredColumns22
print(dataUrl)
data = pd.read_json(dataUrl)
print(data.shape)
data.dtypes

Oops - we see that the water and energy use columns are objects, not float values, which will cause problems. This is because some rows have "Not Available" as a value instead of a number. So let's fix that now.

In [ ]:
# let's fix the data type of the water_use_all_water_sources column - this will also remove all the "Not Available" values
data.water_use_all_water_sources = pd.to_numeric(data.water_use_all_water_sources, errors = 'coerce')

# now let's fix the data type of the indoor_water_use_all_water column
data.indoor_water_use_all_water = pd.to_numeric(data.indoor_water_use_all_water, errors = 'coerce')

# now let's fix the data type of the site_energy_use_kbtu column
data.site_energy_use_kbtu = pd.to_numeric(data.site_energy_use_kbtu, errors = 'coerce')

# now let's fix the data type of the site_energy_use_kbtu column
data.property_gfa_calculated = pd.to_numeric(data.property_gfa_calculated, errors = 'coerce')

data.dtypes

Let's do a simple bar chart that shows water usage by borough.

In [ ]:

# now get the usage by borough. We need to group the data by borough, summing up the values in the "water_use_all_water_sources" column
# we'll also sort the data in descending order.
usage_by_borough = data.groupby('borough').water_use_all_water_sources.sum().sort_values(ascending=False)

# now plot
usage_by_borough.iplot(kind='bar', title='Water Usage')


While it's great to be able to plot a chart with a single line of code, this chart does not look very nice and also doesn't follow best practices. The chart is too wide, max aspect ratio for a bar chart should be 1:1.5 in otherwords the dimensions should be 600px high by 900 wide for example
-	The title should be centred and be more descriptive “2022 water usage by NYC Borough” 
-	The X and Y axis have no labels
-	The column colours are all the same, Manhattan is more distinctive. Let's put Manhattan in dark blue and all the rest in medium grey 

Here's more robust code that implements these formatting changes:

In [ ]:
filtered = data[['borough', 'water_use_all_water_sources']]
usage_by_borough = filtered.groupby('borough').sum(['water_use_all_water_sources']).sort_values(by='water_use_all_water_sources', ascending=False).reset_index().rename(columns={0: "borough"})

colors = ['lightslategray',] * 5
colors[0] = 'blue'
usage = go.Bar(
    x=usage_by_borough.borough,
    y=usage_by_borough.water_use_all_water_sources,
    marker_color=colors
)
layout = go.Layout(
        width=900,
        height=600,
        xaxis=dict(title_text="Borough", color="slategray"),
        yaxis=dict(title_text="Water Usage kgal", color="slategray"),
        template='simple_white',
        title=dict(text="2022 water usage by NYC Borough", x=.5),
        font=dict(family="Rockwell"),
        title_font=dict(size=24),
)
fig = go.Figure(data=usage, layout=layout, skip_invalid=False)
fig


In [ ]:
# now let's compare indoor vs outdoor water sources. We don't explicitly have outdoor water
# so we need to create the column

# we don't need all the other columns for this analysis - so let's just take the columns we need
filtered = data[['borough', 'water_use_all_water_sources', 'indoor_water_use_all_water']]

# remove unlabeled boroughs, rows without water metrics
filtered = filtered[~filtered.borough.isna() & ~filtered['water_use_all_water_sources'].isna()  & ~filtered['indoor_water_use_all_water'].isna()]

# let's check to see how many rows we filtered... we started with 13k
filtered.shape

10k of the rows in this set don't have complete water usage data. But 19k rows remaining is still good for trends.

In [ ]:

# let's group by borough and sum the water_use_all_water_sources and indoor_water_use_all_water columns
usage_by_borough = filtered.groupby('borough').sum(['water_use_all_water_sources', 'indoor_water_use_all_water']).sort_values(by='water_use_all_water_sources', ascending=False)

# now let's create a new column for outdoor usage by subtracting indoor from the total
usage_by_borough['outdoor_water_use_all_water'] = (usage_by_borough.water_use_all_water_sources - usage_by_borough.indoor_water_use_all_water)

# now let's create a stacked bar graph with the indoor & outdoor usage per borough
# the "text" parameter calculates the relative % and puts that as an additional text labelw
colors = ['lightslategray',] * 5
colors[0] = 'blue'
indoor = go.Bar(
    x = usage_by_borough.index,
    y = usage_by_borough['indoor_water_use_all_water'],
    name = 'Indoor Usage', 
    text=(100 * usage_by_borough['indoor_water_use_all_water'] / usage_by_borough['water_use_all_water_sources']).apply(lambda x: '{0:1.2f}%'.format(x)),
    marker_color=colors 
)
outdoor = go.Bar(
    x=usage_by_borough.index, 
    y=usage_by_borough['outdoor_water_use_all_water'],
    name="Outdoor Usage",
    text=(100 * usage_by_borough['outdoor_water_use_all_water'] / usage_by_borough['water_use_all_water_sources']).apply(lambda x: '{0:1.2f}%'.format(x)),
)
graphdata = [indoor, outdoor]
layout = go.Layout(
        width=900,
        height=600,
        xaxis=dict(title_text="Borough", color="slategray"),
        yaxis=dict(title_text="Water Usage kgal", color="slategray"),
        template='simple_white',
        title=dict(text="2022 water usage by NYC Borough", x=.5),
        font=dict(family="Rockwell"),
        title_font=dict(size=24),
        barmode="stack"
)

fig = go.Figure(data=graphdata, layout=layout)
fig

This graph tells us that the distinction between indoor usage and overall usage is not terribly valuable in this data set - even the "complete" data is clearly not accurate, as it's unlikely there is 0 outdoor usage in the bronx. So for now we will just focus on overall water consumption and add back in the rows with missing indoor stats.

Next let's look at energy consumption to see if there's a correlation to water consumption. For this analysis, we'll create a scatter plot diagram of Water vs Energy usage grouped by Borough.

In [ ]:
# we don't need all the other columns for this analysis - so let's just take the columns we need
filtered = data[['borough', 'water_use_all_water_sources', 'site_energy_use_kbtu']]

# remove unlabeled boroughs, rows without water metrics
filtered = filtered[~filtered.borough.isna() & ~filtered.water_use_all_water_sources.isna() & ~filtered.site_energy_use_kbtu.isna()]

# how many rows?
filtered.shape


In [ ]:

# let's group by borough and sum the water_use_all_water_sources and indoor_water_use_all_water columns
usage_by_borough = filtered.groupby('borough').sum(['water_use_all_water_sources', 'site_energy_use_kbtu']).sort_values(by='water_use_all_water_sources', ascending=False)

# now let's create a combo graph with the water usage per borough as a bar and the energy as a line
water = go.Scatter(
    y = usage_by_borough['site_energy_use_kbtu'],
    x = usage_by_borough['water_use_all_water_sources'],
    text=usage_by_borough.index,
    mode='markers',
    marker_size=20,
    name = 'Water Usage', 

)
layout = go.Layout(
        width=900,
        height=600,
        xaxis=dict(title_text="Water Usage kgal", color="slategray"),
        yaxis=dict(title_text="Energy Usage kbtu", color="slategray"),
        template='simple_white',
        title=dict(text="2022 water usage vs energy usage by NYC Borough", x=.5),
        font=dict(family="Rockwell"),
        title_font=dict(size=24),
)

graphdata = [water]
fig = go.Figure(data=graphdata, layout=layout)
fig

Looks like there's a linear relationship between water and energy usage. Let's draw a trendline using an ordinary least-squares linear regression model. We will need to switch to plot.ly express for this, as the graph_objects library doesn't have built in trendlines.

In [ ]:
fig = px.scatter(
    graphdata, 
    y = usage_by_borough['site_energy_use_kbtu'],
    x = usage_by_borough['water_use_all_water_sources'], 
    trendline="ols",
    title='Water vs Energy Usage by Borough',
    labels={'x':'Water Use (all sources)', 'y':'Energy Use (kBTU)'}
)
fig.update_layout({
    'width': 900,
    'height':600,
    'plot_bgcolor': 'rgba(255, 255, 255, 0)',
    'paper_bgcolor': 'rgba(255, 255, 255, 255)',
    'font': dict(family="Rockwell", color='slategray'),
    'title_font':dict(size=24, family="Rockwell",color='black'),
    'title_x':0.5,
    'autosize':True,
    'xaxis': dict(
        visible=True,
     
    ),
    'yaxis': dict(
        visible=True,
      
    )
})
fig

This graph doesn't tell us as much as it looks like it does, because it's not adjusted for population.

What we need to do is adjust the data series based on occupancy and/or population. We don't have population statistics in this data set but we do have occupancy.

In [ ]:
# we don't need all the other columns for this analysis - so let's just take the columns we need
filtered = data[['borough', 'water_use_all_water_sources', 'site_energy_use_kbtu', 'occupancy']]

# remove unlabeled boroughs, rows without metrics
filtered = filtered[~filtered.borough.isna() & ~filtered.occupancy.isna() & ~filtered.water_use_all_water_sources.isna() & ~filtered.site_energy_use_kbtu.isna()]

# how many rows?
filtered.shape

In [ ]:
# graph
# let's group by borough and sum the water_use_all_water_sources and indoor_water_use_all_water columns
# usage_by_borough = filtered.groupby('borough').sum(['percap_water', 'percap_energy']).sort_values(by='percap_water', ascending=False)
usage_by_borough = filtered.groupby('borough').sum(['water_use_all_water_sources', 'site_energy_use_kbtu', 'occupancy']).reset_index()
usage_by_borough['percap_water'] = usage_by_borough['water_use_all_water_sources'] / usage_by_borough['occupancy']
usage_by_borough['percap_energy'] = usage_by_borough['site_energy_use_kbtu'] / usage_by_borough['occupancy']
usage_by_borough['combined'] = usage_by_borough['percap_water'] + (usage_by_borough['percap_energy'] / 10000)
usage_by_borough['ranked'] = usage_by_borough['combined'].rank()
usage_by_borough['size'] = usage_by_borough['occupancy'].rank()
usage_by_borough = usage_by_borough.sort_values('ranked', ascending=False)
# let's look at the data set now
usage_by_borough

These numbers look reasonable and in roughly in the same ranges, so let's graph to see if anything jumps out at us.

In [ ]:

# make sure x & y axis starts at 0!
water = go.Scatter(
    y = usage_by_borough['percap_energy'],
    x = usage_by_borough['percap_water'],
    text=usage_by_borough['borough'],
    mode='markers',
    marker=dict(
        size=usage_by_borough['size']  *  20,
    ),
    name = 'Borough', 
    showlegend=True
)

layout = go.Layout(
        width=900,
        height=600,
        xaxis=dict(title_text="Water Usage kgal", color="slategray"),
        yaxis=dict(title_text="Energy Usage kbtu", color="slategray"),
        template='simple_white',
        title=dict(text="2022 water usage vs energy usage by NYC Borough", x=.5),
        font=dict(family="Rockwell"),
        title_font=dict(size=24),
)

graphdata = [water]
fig = go.Figure(data=graphdata, layout=layout)
fig.show()



OK - this is interesting - we can see that Staten Island is an outlier in terms of per-capita water and energy usage. We'll want to dig into that more later. But for now we can still solve some issues with this graph. 

1. The x and y axis don't start at 0 - this creates a distorted sense of scale.
1. All the boroughs are the same color - we should highlight the outlier in blue and make the rest gray.

As noted before, charting for the purpose of Exploratory Data Analysis (EDA) is different than charting for the purpose of communication. When formatting a chart for presentation, we'll often need to use very granular charting code with "hardcoded" values, such as calling out a particular series. To clean up this chart, our charting code is going to need to deal with each bubble plot individually. 

In [ ]:
colors = ['lightslategray',] * 5
colors[0] = 'blue'

import plotly.graph_objects as go
fig = go.Figure(data=go.Scatter(
    x=usage_by_borough.query("borough == 'STATEN IS'")['percap_water'],
    y=usage_by_borough.query("borough == 'STATEN IS'")['percap_energy'],
    name="Staten Island",
    mode='markers',
    marker=dict(
        size=usage_by_borough.query("borough == 'STATEN IS'")['size']  *  20,
        color=colors[0]
    )
))

boroughs = ["MANHATTAN", "BRONX", "BROOKLYN", "QUEENS"]
for borough in boroughs: 
    fig.add_trace(go.Scatter(
        x = usage_by_borough.query("borough == '" + borough + "'")['percap_water'],
        y = usage_by_borough.query("borough == '" + borough + "'")['percap_energy'],
        name = borough,
        mode='markers',
        marker=dict(
            size=usage_by_borough.query("borough  == '" + borough + "'")['size']  *  20,
            color=colors[1]
        ),
    ))

annotations = [
    dict(
        x=x, 
        y=y,
        text=z.title(), # Some conditional to define outliers
        showarrow=True,
        xshift = -2 * size,
        yshift = 10 * size,
        # xanchor='center',  # Position of text relative to x axis (left/right/center)
        # yanchor='middle',  # Position of text relative to y axis (top/bottom/middle)
    ) for x, y, z, size in zip(usage_by_borough['percap_water'], usage_by_borough['percap_energy'], usage_by_borough['borough'], usage_by_borough['size'])
]

fig.update_layout(go.Layout(
        xaxis=dict(title_text="Per-Capita Water Usage kgal", color="slategray"),
        yaxis=dict(title_text="Per-Capita Energy Usage kbtu", color="slategray"),
        title=dict(text="Per-Capita 2022 Water vs Energy Usage by NYC Borough", x=.5),
        font=dict(family="Rockwell"),
        title_font=dict(size=24),
        annotations=annotations,
        showlegend=False
))
fig.update_yaxes(rangemode="tozero")
fig.update_xaxes(rangemode="tozero")
fig.show()

Oh - the chart seems to have lost some of its formatting! Can you go back and fix these issues:
- apply the `simple_white` theme so that the background is white
- set the width and height so the aspect ratio is 1.5:1 (hint: 900x600)

This looks interesting, but really doesn't tell us a lot. It suggests that Staten Island is very inefficient in regard to energy and water usage relative to the other boroughs. But the sample size for Staten Island is small and the other factor is that this data set only looks at large buildings. Staten Island is more residential amd single or dual family homes don't appear in the data.

It is worth exploring Brooklyn and the Bronx's relative energy efficiency, as well as the relatively high per-capita water usage in the Bronx. 

# Exploring the data by property type
To further understand this we will need to unpack the data by property type.

In [ ]:
# we don't need all the other columns for this analysis - so let's just take the columns we need
filtered = data[['primary_property_type', 'water_use_all_water_sources', 'site_energy_use_kbtu', 'occupancy']]

# remove unlabeled boroughs, rows without metrics
filtered = filtered[~filtered.primary_property_type.isna() & ~filtered.occupancy.isna() & ~filtered.water_use_all_water_sources.isna() & ~filtered.site_energy_use_kbtu.isna()]

# how many rows?
filtered.shape


In [ ]:
# graph
# let's group by borough and sum the water_use_all_water_sources and indoor_water_use_all_water columns
# usage_by_borough = filtered.groupby('borough').sum(['percap_water', 'percap_energy']).sort_values(by='percap_water', ascending=False)
usage_by_property = filtered.groupby('primary_property_type').sum(['water_use_all_water_sources', 'site_energy_use_kbtu', 'occupancy'])
usage_by_property['percap_water'] = usage_by_property['water_use_all_water_sources'] / usage_by_property['occupancy']
usage_by_property['percap_energy'] = usage_by_property['site_energy_use_kbtu'] / usage_by_property['occupancy']

# let's look at the data set now - first by overall water usage
usage_by_property.nlargest(10, 'water_use_all_water_sources')


In [ ]:
# now by overall energy usage
usage_by_property.nlargest(10, 'site_energy_use_kbtu')

There are several property types that are common to both lists. Apartment blocks are the top of each; but they also represent 10x the numbrer of people. Offices, Hotels, K-12 schools and wastewater treatment are all high up as well.

Let's take the top 20 by overall water usage and rank them by per-capita usage:

In [ ]:
usage_by_property.nlargest(20,'water_use_all_water_sources').sort_values('percap_water', ascending=False)

And we can do the same for energy:

In [ ]:
usage_by_property.nlargest(20,'site_energy_use_kbtu').sort_values('percap_energy', ascending=False)

There's a lot to see in just the numbers. We can see that a few property types are large consumers of both water and energy no matter how you slice it - overall or per-capita. Wastewater treatment, Hospitals, Hotels, Medical Offices, Offices,  Senior Care Communities.

This gives us our first idea as to a focus area - healthcare seems to be a very water and energy intensive field, and will only grow as our population ages and lives longer. Thinking about conservation that targets the medical sector is great.

It also flags up wastewater treatment as an area that would benefit from innovation that improves efficiencies.

Finally, if you're wondering about the high per-capita energy usage in Prisons, you should know there is no death penalty in NY state, so it is not because of electric chairs.

Let's rank facility types from worst to best by the following formula that prioritizes total usage of water and electricity but also considers per-capita usage

rank for total water + rank for total energy + 50% (rank for per capita water + rank for per capita energy)

We'll create a ranking column and sort by that.

In [ ]:
usage_by_property = filtered.groupby('primary_property_type').sum(['water_use_all_water_sources', 'site_energy_use_kbtu', 'occupancy'])
usage_by_property['percap_water'] = usage_by_property['water_use_all_water_sources'] / usage_by_property['occupancy']
usage_by_property['percap_energy'] = usage_by_property['site_energy_use_kbtu'] / usage_by_property['occupancy']
usage_by_property['water_rank'] = usage_by_property.percap_water.rank()
usage_by_property['energy_rank'] = usage_by_property.percap_energy.rank()
usage_by_property['combined'] = usage_by_property['water_rank'] + usage_by_property['energy_rank']
usage_by_property['ranked'] = usage_by_property['combined'].rank(ascending=False)
usage_by_property['size'] = ((usage_by_property['water_use_all_water_sources'] / 1000) + (usage_by_property['site_energy_use_kbtu'] / 1000000))
usage_by_property.reset_index().nsmallest(20, 'ranked')

Now that we have this data ranked, let's come up with a visualization that helps us understand the energy and water utility of these facilities. 

In [ ]:
ranked = usage_by_property.nsmallest(10, 'ranked')
fig = px.scatter(data_frame=ranked,
                 x="percap_water",
                 y="percap_energy",
                 size="size",
                 color=ranked.index,
                 text=ranked.index,
                 width=900,
                 height=600,
                 title="Per-Capita Energy and Water Usage by Property Type - size is overall usage",
                 labels={"percap_water": "Water Use (Per-Capita)",
                         "percap_energy": "Energy Use (Per-Capita)",
                         "occupancy": "Occupancy",
                         "primary_property_type": "Property Type",
                         "size": "Overall usage of energy and water"},
                 log_x=False,
                 log_y=False)

fig.update_traces(textposition="top center")
fig.show()


This graph is interesting but ugly and confusing. Let's use visualization best-practices to clean it up.

1. Let's reduce the # of items to 5 - processing 10 is too much, especially as the smaller 5 are clustered in the bottom left.
1. Let's improve labelling, fonts, colors and set the background to white.

In [ ]:
colors = ['lightslategray',] * 10
# we will highlight the first two in blue as they are the largest and most interesting data points
colors[0] = 'blue'
colors[1] = 'blue'

ranked = usage_by_property.nsmallest(5, 'ranked')

fig = go.Figure()

properties = ranked.index
count = 0
for property in properties: 
    fig.add_trace(go.Scatter(
        x = ranked.query("primary_property_type == '" + property + "'")['percap_water'],
        y = ranked.query("primary_property_type == '" + property + "'")['percap_energy'],
        name = property,
        mode='markers',
        marker=dict(
            size=ranked.query("primary_property_type  == '" + property + "'")['size'] / 100,
            color=colors[count]
        ),
    ))
    count+=1

annotations = [
    dict(
        # xref="x", yref='y',
        axref="pixel", ayref="pixel",
        x = x,
        y = y,
        ax = (size / 200),
        ay = -20 - (size / 200),
        text=z.title(), # Some conditional to define outliers
        showarrow=True,
        standoff=size / 200,
        xshift = 0, #2 * size / 400,
        yshift = 0, # 20 + (size / 200),
        xanchor='center',  # Position of text relative to x axis (left/right/center)
        yanchor='middle',  # Position of text relative to y axis (top/bottom/middle)
    ) for x, y, z, size in zip(ranked['percap_water'], ranked['percap_energy'], properties, ranked['size'])
]

fig.update_layout(go.Layout(
        width=900,
        height=600,
        xaxis=dict(title_text="Per-Capita Water Usage kgal", color="slategray"),
        yaxis=dict(title_text="Per-Capita Energy Usage kbtu", color="slategray"),
        template='simple_white',
        title=dict(text="Per-Capita 2022 Water vs Energy Usage by Property Type", x=.5),
        font=dict(family="Rockwell"),
        title_font=dict(size=24),
        annotations=annotations,
        showlegend=False
))
fig.update_yaxes(rangemode="tozero")
fig.update_xaxes(rangemode="tozero")
fig.show()

# Hospital Deep-Dive

We can see that hospitals are a major user of energy per-capita. So let's filter our data set to just hospitals and look at the usage per borough:

In [ ]:
# we don't need all the other columns for this analysis - so let's just take the columns we need
filtered = data[['borough', 'primary_property_type', 'water_use_all_water_sources', 'site_energy_use_kbtu', 'occupancy', 'property_gfa_calculated']]

# remove all rows that are not "Hospitals", unlabeled boroughs, rows without metrics
filtered = filtered[filtered.primary_property_type.eq('Hospital (General Medical & Surgical)') & ~filtered.borough.isna() & ~filtered.occupancy.isna() & ~filtered.water_use_all_water_sources.isna() & ~filtered.site_energy_use_kbtu.isna()]

# let's check the data
filtered


This is interesting - the occupancy value for almost every hospital is 100. That seems like an issue in data collection, because most hospitals in NYC have more than 100 beds, let alone 100 "occupants" at any time. Based on that fact, we shouldn't use the occupancy figure to calculate a per-capita usage of energy or water directly.

Anticipating this, we added an extra column to the filtered data - property_gfa_calculated, which is an estimate of the gross square feet of property. Hospitals have reasonably similar designs and space utilization, so this could be a good proxy for the # of people in the hospital. Let's use this to create the same usage-by-borough graph as before, but just for hospitals, and using square feet instead of occupancy.

In [ ]:
# graph
# let's group by borough and sum the water_use_all_water_sources and indoor_water_use_all_water columns
# usage_by_borough = filtered.groupby('borough').sum(['percap_water', 'percap_energy']).sort_values(by='percap_water', ascending=False)
usage_by_borough = filtered.groupby('borough').agg({'occupancy':'count', 'water_use_all_water_sources': 'sum', 'site_energy_use_kbtu': 'sum', 'property_gfa_calculated': 'sum'}).reset_index().rename(columns={'occupancy':'count'})
usage_by_borough['persqf_water'] = usage_by_borough['water_use_all_water_sources'] / usage_by_borough['property_gfa_calculated']
usage_by_borough['persqf_energy'] = usage_by_borough['site_energy_use_kbtu'] / usage_by_borough['property_gfa_calculated']
usage_by_borough['combined'] = usage_by_borough['persqf_water'] + (usage_by_borough['persqf_energy'] / 10000)
usage_by_borough['ranked'] = usage_by_borough['combined'].rank()
usage_by_borough = usage_by_borough.sort_values('ranked', ascending=False)
usage_by_borough


In [ ]:
# THIS CODE INTENTIONALLY HAS AN ERROR - THE WRONG BUBBLE IS COLORED BLUE. FIX IT!

colors = ['lightslategray',] * 5
colors[0] = 'blue'

import plotly.graph_objects as go
fig = go.Figure(data=go.Scatter(
    x=usage_by_borough.query("borough == 'MANHATTAN'")['persqf_water'],
    y=usage_by_borough.query("borough == 'MANHATTAN'")['persqf_energy'],
    name="Staten Island",
    mode='markers',
    marker=dict(
        size=usage_by_borough.query("borough == 'MANHATTAN'")['count'] * 10,
        color=colors[0]
    )
))

boroughs = ["STATEN IS", "BRONX", "BROOKLYN", "QUEENS"]
for borough in boroughs: 
    fig.add_trace(go.Scatter(
        x = usage_by_borough.query("borough == '" + borough + "'")['persqf_water'],
        y = usage_by_borough.query("borough == '" + borough + "'")['persqf_energy'],
        name = borough,
        mode='markers',
        marker=dict(
            size=usage_by_borough.query("borough  == '" + borough + "'")['count'] * 10,
            color=colors[1]
        ),
    ))

annotations = [
    dict(
        x=x, 
        y=y,
        text=z.title(), # Some conditional to define outliers
        showarrow=True,
        xshift = -2 * (size / 2),
        yshift = 10 * (size / 2),
        # xanchor='center',  # Position of text relative to x axis (left/right/center)
        # yanchor='middle',  # Position of text relative to y axis (top/bottom/middle)
    ) for x, y, z, size in zip(usage_by_borough['persqf_water'], usage_by_borough['persqf_energy'], usage_by_borough['borough'], usage_by_borough['count'])
]

fig.update_layout(go.Layout(
        width=900,
        height=600,
        xaxis=dict(title_text="Per-Sq Ft Water Usage kgal/sqf", color="slategray"),
        yaxis=dict(title_text="Per-Sq Ft Energy Usage kbtu/sqf", color="slategray"),
        template='simple_white',
        title=dict(text="Per-Sq Ft 2022 Water vs Energy Usage for Hospitals by NYC Borough <br><sup>Bubble size shows # of hospitals</sup>", x=.5),
        font=dict(family="Rockwell"),
        title_font=dict(size=24),
        annotations=annotations,
        showlegend=False
))
fig.update_yaxes(rangemode="tozero")
fig.update_xaxes(rangemode="tozero")
fig.show()

Did you fix the chart so that Staten Island is blue instead of Manhattan? If not, go back and try again!

We can see that for all the boroughs except Staten Island, the water and energy usage is roughly the same based on square feet of hospital space. However Staten Island is a complete outlier, using significantly more water and energy per square foot of hospital space than anywhere else in NYC. This would be worth exploring in greater detail.

# Your Excersise

Now it's your turn. Let's do the same analysis for Waste-Water treatment. You need to decide whether to use occupancy, square footage or some other factor for comparison. 
